In [1]:
import pandas as pd

In [2]:
base_dir = 'car_sales_data'

In [3]:
import glob

In [9]:
shape = 0

for file in glob.glob(f'{base_dir}/*.csv'):
    car_sales = pd.read_csv(file)
    shape += car_sales.shape[0]

shape

12899990

In [10]:
# Create a data frame to store
# sales count by state

sales_count_by_state = pd.DataFrame(columns=['state', 'sale_count'])

for file in glob.glob(f'{base_dir}/*.csv'):
    car_sales = pd.read_csv(file)
    sale_count_by_state = car_sales.groupby('state').size().reset_index(name='sale_count')
    sales_count_by_state = pd.concat([sales_count_by_state, sale_count_by_state])


In [11]:
import polars as pl

In [12]:
# Read all the files
car_sales = pl.read_csv(f'{base_dir}/*.csv')

In [13]:
car_sales.shape

(12899990, 15)

In [14]:
# Get sales by state using polars
sales_by_state = car_sales.group_by('state').agg(
    pl.sum('sale_price').alias('total_sales'),
    pl.count('sale_price').alias('sales_count')
)

In [15]:
sales_by_state.shape

(10, 3)

In [16]:
pl.Config.set_fmt_float("full")

polars.config.Config

In [17]:
sales_by_state.head()

state,total_sales,sales_count
str,f64,u32
"""PA""",48412069561.59,1289814
"""FL""",48361954451.31,1289117
"""NC""",48410710436.63,1290735
"""GA""",48347024626.96,1288599
"""OH""",48412988663.34,1290729


In [18]:
sales_by_state.sort('total_sales', descending=True)

state,total_sales,sales_count
str,f64,u32
"""TX""",48429721750.82,1290867
"""NY""",48417155335.06,1290693
"""OH""",48412988663.34,1290729
"""PA""",48412069561.59,1289814
"""NC""",48410710436.63,1290735
"""IL""",48399698308.21,1289672
"""CA""",48394059754.01,1289455
"""MI""",48389009099.38,1290309
"""FL""",48361954451.31,1289117


In [19]:
from pyspark.sql import SparkSession

In [20]:
session = SparkSession. \
    builder. \
    appName("Car Sales"). \
    getOrCreate()

26/01/03 08:47:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [22]:
car_sales = session. \
    read. \
    csv(base_dir, 
        header=True, 
        inferSchema=True
    )

In [23]:
car_sales.count()

12899990

In [24]:
from pyspark.sql.functions import count, col

In [25]:
sales_by_state = car_sales.groupBy('state').agg(
    count('sale_price').alias('sales_count')
)

In [27]:
sales_by_state.orderBy(col('sales_count').desc())

DataFrame[state: string, sales_count: bigint]

In [28]:
sales_by_state.show()

+-----+-----------+
|state|sales_count|
+-----+-----------+
|   MI|    1290309|
|   CA|    1289455|
|   NC|    1290735|
|   IL|    1289672|
|   OH|    1290729|
|   PA|    1289814|
|   NY|    1290693|
|   TX|    1290867|
|   GA|    1288599|
|   FL|    1289117|
+-----+-----------+

